# Introduction:

For this Quantative Risk Assesment (QRM) assignement, we will be comparing and quantativly assesing the usability of a Large Language Model (LLM) - in this case Llama3 by Meta - when implemented in a company use case.

### The Case
The project case focuses on evaluating the risks and benefits of using LLMs to enhance web-scrapers/AI-powered asistants, specifically by comparing their performance with and without Retrieval-Augmented Generation (RAG). The goal is to quantify how RAG reduces risks such as hallucination and data inaccuracies, using metrics like Error Rate (ER), and to assess the trade-offs in implementing this approach in a quantitative risk management framework.

### The Model
Llama 3 by Meta is a transformer-based Large Language Model designed for natural language understanding and generation. It has a data cut-off point from mid-2023, meaning it is trained on datasets available up until that period. Llama 3 is optimized for high efficiency, offering a balance between computational demands and performance, and it supports integration with external tools such as Retrieval-Augmented Generation (RAG) to enhance its ability to work with up-to-date or domain-specific information beyond its training data.

### The Data
For our data, we will use the AI-act from the EU, published in 2024. The EU AI Act dataset is particularly suitable for this project because it was published after Llama 3’s data cut-off point (mid-2023). This ensures that the model has not been trained on this document, providing an ideal scenario to evaluate the model's ability to interpret and summarize new, unseen data. 

Furthermore, the EU AI Act contains complex and structured information, making it a robust test for assessing the model’s performance in understanding and accurately representing legal and technical content. This setup allows for testing how Retrieval-Augmented Generation (RAG) can improve accuracy and reduce hallucinations when integrated. 

For the LLM to understand the information, the data will be indexed into a vector database to make it searchable by the RAG system.

## Installation and Imports

In [ ]:
!pip install -r requirements.txt

In [1]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

/work/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [2]:
#model_id = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
#model_id = '/work/QRM/Llama-3.1-8B/original'
model_id = "meta-llama/Llama-3.1-8B"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# using the `bitsandbytes` library
# we set quantization configuration to load large model with less GPU memory
# possibly not needed with larger setups
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


Now we prepare the tokenizer and the model. To gain a sense of how long these steps will take, we wrap the pipeline in a timer. 

OBS! Be aware that running this model without a dedicated GPU won't be possible. To run this project I've used UCloud computers with  that were allocated to me for this project.
You will also need access to the model from Huggingface at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct and use the command `huggingface-cli login` to provide your token for this download to work. Other methods are possible to download and use the model from Meta, but this was the way I did it. 

In [4]:
time_start = time()

model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024 #can be changed as needed
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


Prepare model, tokenizer: 389.966 sec.



We next define the query pipeline.

To ensure proper functionality when setting up the HuggingFace pipeline, we especially need to specify the `max_length` here. This prevents the pipeline from defaulting to the very short length of 20 which we can't use for our case.

In [5]:
time_start = time()

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_new_tokens=1000, #ajust as needed
        device_map="auto",)

time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

Prepare pipeline: 0.614 sec.


We should now test our pipeline by defining a function

In [6]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """    
    time_start = time()
    
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"

## Testing our pipeline and model

We test the pipeline using a few queries related to the European Union Artificial Intelligence Act (EU AI Act).

Additionally, we create a UI-utility function to neatly display the LLM's output. This function includes the calculation time, the input question, and the generated answer, all formatted for clear and easy readability. This will help us later when we aim to compare the output between the model with and without RAG.

In [6]:
from IPython.display import display, Markdown

def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

### Testing

Let's test the model with a few questions regarding the AI-act. 

Later we will build a repo of all questions and save the responses in seperate columns for RAG and non-RAG responses.

As an example, we will ask `What is the purpose of the EU AI regulation act?`. Here we are looking for the following from page 1 of the act: 

<I>"The purpose of this Regulation is to improve the functioning of the internal market by laying down a uniform legal 
framework in particular for the development, the placing on the market, the putting into service and the use of 
artificial intelligence systems (AI systems) in the Union, in accordance with Union values, to promote the uptake of 
human centric and trustworthy artificial intelligence (AI) while ensuring a high level of protection of health, safety, 
fundamental  rights  as  enshrined  in  the  Charter  of  Fundamental  Rights  of  the  European  Union  (the  ‘Charter’), 
including democracy, the rule of  law and environmental protection, to protect against the harmful effects of AI 
systems  in  the  Union,  and  to  support  innovation.  This  Regulation  ensures  the  free  movement,  cross-border,  of 
AI-based  goods  and  services,  thus  preventing  Member  States  from  imposing  restrictions  on  the  development, 
marketing and use of AI systems, unless explicitly authorised by this Regulation."</I>

In [12]:
response = test_model(tokenizer,
                    query_pipeline,
                   "What is the purpose of the EU AI regulation act?")
display(Markdown(colorize_text(response)))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




**<font color='red'>Question:</font>** What is the purpose of the EU AI regulation act?


**<font color='green'>Answer:</font>**  What does it mean for the future of AI? And how does it affect the AI industry? In this blog post, we will explore the purpose of the EU AI regulation act and its impact on the AI industry. We will also discuss the benefits and drawbacks of this regulation and how it will affect the future of AI.
What is the purpose of the EU AI regulation act?
The EU AI regulation act is a set of rules and regulations that govern the development, deployment, and use of artificial intelligence (AI) systems within the European Union. The purpose of this act is to ensure that AI systems are developed and deployed in a safe and ethical manner, and that they respect the fundamental rights of individuals.
The EU AI regulation act sets out a number of principles that must be followed when developing and deploying AI systems. These principles include:
The EU AI regulation act also sets out a number of requirements that must be met by AI systems. These requirements include:
The EU AI regulation act is intended to ensure that AI systems are developed and deployed in a safe and ethical manner, and that they respect the fundamental rights of individuals. It is also intended to promote innovation and competitiveness in the EU’s AI sector.
What does it mean for the future of AI?
The EU AI regulation act has the potential to shape the future of AI in a number of ways. Firstly, it sets out a number of principles that must be followed when developing and deploying AI systems. These principles include:
The EU AI regulation act also sets out a number of requirements that must be met by AI systems. These requirements include:
The EU AI regulation act is intended to ensure that AI systems are developed and deployed in a safe and ethical manner, and that they respect the fundamental rights of individuals. It is also intended to promote innovation and competitiveness in the EU’s AI sector.
The EU AI regulation act could have a number of benefits for the future of AI. Firstly, it could help to ensure that AI systems are developed and deployed in a safe and ethical manner. This could help to prevent accidents and other negative outcomes that could result from the use of AI systems.
Secondly, the EU AI regulation act could help to promote innovation and competitiveness in the EU’s AI sector. This could help to ensure that the EU remains a leader in the development and deployment of AI systems.
Thirdly, the EU AI regulation act could help to ensure that AI systems respect the fundamental rights of individuals. This could help to prevent the misuse of AI systems, such as the use of AI systems to discriminate against individuals or to violate their privacy rights.
The EU AI regulation act could also have some drawbacks for the future of AI. Firstly, it could be seen as a barrier to innovation and competitiveness in the EU’s AI sector. This could discourage companies from developing and deploying AI systems in the EU.
Secondly, the EU AI regulation act could be seen as a barrier to the development of new AI technologies. This could discourage companies from developing and deploying new AI technologies in the EU.
Thirdly, the EU AI regulation act could be seen as a barrier to the use of AI systems in certain sectors, such as healthcare or finance. This could discourage companies from developing and deploying AI systems in these sectors.
Overall, the EU AI regulation act could have both benefits and drawbacks for the future of AI. It is important to consider both the benefits and drawbacks when deciding whether or not to support the EU AI regulation act.
How does it affect the AI industry?
The EU AI regulation act could have a number of impacts on the AI industry. Firstly, it could help to ensure that AI systems are developed and deployed in a safe and ethical manner. This could help to prevent accidents and other negative outcomes that could result from the use of AI systems.
Secondly, the EU AI regulation act could help to promote innovation and competitiveness in the EU’s AI sector. This could help to ensure that the EU remains a leader in the development and deployment of AI systems.
Thirdly, the EU AI regulation act could help to ensure that AI systems respect the fundamental rights of individuals. This could help to prevent the misuse of AI systems, such as the use of AI systems to discriminate against individuals or to violate their privacy rights.
The EU AI regulation act could also have some impacts on the AI industry. Firstly, it could be seen as a barrier to innovation and competitiveness in the EU’s AI sector. This could discourage companies from developing and deploying AI systems in the EU.
Secondly, the EU AI regulation act could be seen as a barrier to the development of new AI technologies. This could discourage companies from developing and deploying new AI technologies in the EU.
Thirdly, the EU AI regulation act could be seen as a barrier to the use of AI systems in certain sectors, such as healthcare or finance. This could discourage companies from developing and deploying AI systems in these sectors.
Overall, the EU AI regulation act could have both benefits and drawbacks for the AI industry. It is important to


**<font color='magenta'>Total time:</font>** 27.209 sec.

Next we will ask `How many recitals are included in the first section of the EU AI Act document? Please provide the last numbered recital for an exact count`. 

This anwser is quite straightforward, since there are 180 of these recitals. 

In [13]:
response = test_model(tokenizer,
                    query_pipeline,
                   "How many recitals are included in the first section of the EU AI Act document? Please provide the last numbered recital for an exact count")
display(Markdown(colorize_text(response)))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




**<font color='red'>Question:</font>** How many recitals are included in the first section of the EU AI Act document? Please provide the last numbered recital for an exact count


**<font color='green'>Answer:</font>** . Thanks.
The recitals are numbered from 1 to 19.


**<font color='magenta'>Total time:</font>** 0.445 sec.

As can be seen, the answers are not really helpfull or accurate. The model is obviously halucinating since we are asking for information that it has not been trained on or "seen" before.

## Retrival Augmented Generation (RAG)

To improve the accuracy of our model, we should build a RAG system that will allow the model to "read" our document before responding.
To create this RAG system we will be using a HuggingFacePipeline. The overall stepes are as follows:
- Using a `HuggingFacePipline`, we will test the model
- We will then ingest the EU AI act using `PyPdfLoader`
- Sperate or Chunk the AI act into chuncks of 1000 characters, making sure that we have a partial overlap between the chunks of 100 characters.
- Generate embeddings and store the transformed text (processed from the PDF, chunked with overlap, embedded, and indexed) into the vector database.
- Build a `QA_Request` pipeline, incorporating both the retrieval and generation steps.

### Test the model with HuggingFace Pipeline

We will test the model using a HuggingFace pipeline by querying about the meaning of the EU AI Act. By utilizing the HuggingFacePipeline, we will have a more seamless integration with LangChain tasks.

In [7]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

# checking again that everything is working fine
time_start = time()
question = "Please explain what EU AI Act is."
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))

/tmp/ipykernel_1764/3585412621.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=query_pipeline)
/tmp/ipykernel_1764/3585412621.py:6: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(prompt=question)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>** Please explain what EU AI Act is. How can we use it to our advantage?
The EU AI Act is a regulatory framework that aims to address the ethical and societal implications of artificial intelligence (AI) while promoting innovation and competitiveness. It sets out a comprehensive approach to regulating AI systems, covering areas such as transparency, accountability, and safety.
To use the EU AI Act to our advantage, we need to understand its key principles and requirements. The Act emphasizes the need for AI systems to be transparent, explainable, and accountable, ensuring that individuals and organizations can understand how AI systems make decisions and have mechanisms in place to challenge or correct them. Additionally, the Act promotes the development of trustworthy AI systems, encouraging the use of human oversight and ethical considerations in AI design and deployment.
By adhering to the principles and requirements of the EU AI Act, organizations can demonstrate their commitment to responsible AI practices and build trust with their stakeholders. This can lead to increased customer confidence, improved brand reputation, and better relationships with regulatory bodies.
Furthermore, the Act provides a framework for organizations to assess the risks and benefits of AI systems, enabling them to make informed decisions about their use and deployment. This can help organizations avoid potential pitfalls and ensure that their AI systems are safe, secure, and compliant with the Act’s requirements.
Overall, the EU AI Act offers organizations an opportunity to align their AI practices with ethical and societal considerations, while also promoting innovation and competitiveness in the AI space. By understanding and embracing the principles and requirements of the Act, organizations can use it to their advantage, fostering trust, transparency, and responsible AI practices.
How can we use EU AI Act to our advantage?
To use the EU AI Act to our advantage, we need to understand its key principles and requirements. The Act emphasizes the need for AI systems to be transparent, explainable, and accountable, ensuring that individuals and organizations can understand how AI systems make decisions and have mechanisms in place to challenge or correct them. Additionally, the Act promotes the development of trustworthy AI systems, encouraging the use of human oversight and ethical considerations in AI design and deployment.
By adhering to the principles and requirements of the EU AI Act, organizations can demonstrate their commitment to responsible AI practices and build trust with their stakeholders. This can lead to increased customer confidence, improved brand reputation, and better relationships with regulatory bodies.
Furthermore, the Act provides a framework for organizations to assess the risks and benefits of AI systems, enabling them to make informed decisions about their use and deployment. This can help organizations avoid potential pitfalls and ensure that their AI systems are safe, secure, and compliant with the Act’s requirements.
Overall, the EU AI Act offers organizations an opportunity to align their AI practices with ethical and societal considerations, while also promoting innovation and competitiveness in the AI space. By understanding and embracing the principles and requirements of the Act, organizations can use it to their advantage, fostering trust, transparency, and responsible AI practices.
What are the key principles and requirements of EU AI Act?
The key principles and requirements of the EU AI Act include:
  1. Transparency and explainability: AI systems must be transparent and explainable, ensuring that individuals and organizations can understand how AI systems make decisions and have mechanisms in place to challenge or correct them.
  2. Accountability: AI systems must be accountable, with mechanisms in place to ensure that individuals and organizations can hold AI systems responsible for their decisions and actions.
  3. Trustworthy AI: AI systems must be trustworthy, with human oversight and ethical considerations in AI design and deployment.
  4. Safety and security: AI systems must be safe and secure, with measures in place to prevent and mitigate potential risks and harms.
  5. Proportionality: AI systems must be proportionate, with their use and deployment in line with the risks and benefits of the specific application.
  6. Data protection and privacy: AI systems must respect data protection and privacy rights, with appropriate measures in place to protect personal data and ensure privacy.
  7. Human oversight and control: AI systems must have human oversight and control, with mechanisms in place to ensure that human oversight and control are exercised when necessary.
  8. Compliance with other EU laws: AI systems must comply with other EU laws, such as data protection and privacy laws, and must be designed and deployed in a way that respects these laws.
By adhering to these principles and requirements, organizations can demonstrate their commitment to responsible AI practices and build trust with their stakeholders. This can lead to increased customer confidence, improved brand reputation, and better relationships with regulatory bodies.
How can we demonstrate our commitment to responsible AI practices?
To demonstrate our commitment to responsible AI practices, we can take the following steps:
  1. Develop and implement an AI ethics framework: We can develop and implement an AI ethics framework that outlines our principles and requirements for responsible AI practices. This framework should cover areas such as transparency, accountability, safety, and human oversight.
  2. Conduct


**<font color='magenta'>Total time:</font>** 25.604 sec.

We should now ingest our data (EU AI Act). We will be using the `PyPDFLoader` from Langchain. 

In [8]:
loader = PyPDFLoader("./EU_AI_Act.pdf")
documents = loader.load()

### Splitting the data

Using a recursive character text splitter, we will split our data into chunks.
- chunk_size: 1000 (the size of a chunk in characters)
- chunk_overlap: 100 (the size of characters that two chunks overlap)

Chunk overlapping balances chunk size constraints and context preservation, enhancing the effectiveness and accuracy of LLMs in processing text.

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1100, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

### Creating Embeddings with Sentence Transformers and HuggingFace

We generate embeddings using **Sentence Transformer** models and **HuggingFace embeddings**. 

#### Handling Availability Issues
Occasionally, HuggingFace's sentence-transformer models may not be accessible online. To address this, we implement a mechanism that enables the use of locally stored Sentence Transformer models, ensuring uninterrupted functionality even in offline or restricted environments.


In [10]:
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
model_kwargs = {"device": "cuda"}

# try to access the sentence transformers from HuggingFace: https://huggingface.co/api/models/sentence-transformers/all-mpnet-base-v2
try:
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    # alternatively, we will access the embeddings models locally
    local_model_path = "/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

/tmp/ipykernel_1764/682765506.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


### Initializing ChromaDB with Document Splits and Embeddings

We initialize **ChromaDB** using the following:

- **Document Splits:** Pre-processed chunks of the original text.
- **Embeddings:** Generated embeddings from the previously defined models.

#### Enabling Persistence
To ensure data is stored and accessible for future use, we enable the **persistence** option for the vector database, allowing ChromaDB to save data locally.


In [11]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

### Initializing the Chain

We initialize a **QA_Retrival task chain** using LangChain utilities.

#### How It Works
1. **Querying the Vector Database:**  
   The chain begins by querying the vector database using **similarity search** with the provided prompt.

2. **Retrieving Context:**  
   The vector database retrieves relevant documents that match the query.

3. **Composing the Prompt:**  
   The query and the retrieved context are combined to create a prompt. This prompt instructs the LLM to answer the query.

4. **Generating the Response:**  
   The LLM uses the retrieved context to generate an accurate and context-aware response.

This process is known as **Retrieval-Augmented Generation (RAG)** because it combines **retrieval** of relevant data with **generation** of responses based on that data.


In [42]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [13]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate


def format_docs(docs):
    content = "\n\n".join(doc.page_content for doc in docs)
    return content[:3000]  # Limit context length

# Either use a pre-built RAG prompt from LangChain Hub
#prompt = hub.pull("rlm/rag-prompt")

# Or use my custome one here:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Answer in exactly five sentences. Do not write more or fewer sentences.


Question: {question}
Context: {context}
Answer"""
)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,  # Retrieve and format documents
        "question": RunnablePassthrough(),  # Pass through the question
    }
    | prompt                                # Apply the RAG prompt
    | llm                                   # Use the LLM for generation
    | StrOutputParser()                     # Parse the output
)

# Function to clean response
def clean_response(response):
    """
    Extract and return only the concise answer from the LLM's response.
    """
    # Ensure "Answer:" is used as a delimiter to focus on the key part of the response
    if "Answer:" in response:
        response = response.split("Answer:")[-1].strip()
    
    # Remove any extraneous text, keeping the first concise answer
    response = response.split("\n")[0].strip()  # Take only the first meaningful line
    return response

# Query and generate output
query = "What is the purpose of the EU AI regulation act?"
response = qa_chain.invoke(query)
#response = clean_response(response)
print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. Answer in exactly five sentences. Do not write more or fewer sentences.


Question: What is the purpose of the EU AI regulation act?
Context: them are SMEs.
5. By 2 August 2028, the Commission shall evaluate the functioning of the AI Offi ce, whether the AI Office has been 
given sufficient powers and comp etences to fulf il its tasks, and whether it would be relevant and needed f or the proper 
imp lementation and enf orcement of this Regulation to upgrade the AI Offi ce and its enf orcement compet ences and to 
increase its resources. The Commission shall submit a repor t on its evaluation to the European Parliame nt and to the 
Council.
6. By 2 Augu st 2028 and ever y f our y ears thereaf te r , the Commission shall submit a repor t on the review of the progress 
on the development of standardisation del

## Test the RAG

We now define a new test function that will run our query and provide us with a RAG-based response.

In [28]:
def test_rag(qa, query):
    """
    Test the Retrieval Augmented Generation (RAG) system.
    
    Args:
        qa (RetrievalQA.from_chain_type): Langchain function to perform RAG
        query (str): query for the RAG system
    Returns:
        None
    """

    time_start = time()
    #response = qa.run(query)
    response = qa.run({"query": query, "stop": ["\n\n"]})
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [40]:
def test_rag(qa, query):
    from time import time
    from IPython.display import display, Markdown

    time_start = time()

    # Retrieve and deduplicate context
    retrieved_docs = retriever.get_relevant_documents(query)
    unique_docs = list({doc.page_content: doc for doc in retrieved_docs}.values())
    context = " ".join([doc.page_content for doc in unique_docs])

    # Generate response using QA chain
    response = qa.run({"query": query, "context": context})

    # Post-process response to remove repetitions
    def remove_repeated_text(output):
        sentences = output.split('. ')
        unique_sentences = list(dict.fromkeys(sentences))
        return '. '.join(unique_sentences)

    response = remove_repeated_text(response)

    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."

    # Display the final output
    full_response = f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

Now for the fun part - checking our queries:

In [43]:
query = "What is the primary purpose of the EU AI Act according to its introduction?"
test_rag(qa, query)

/tmp/ipykernel_1764/1465479532.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)
/tmp/ipykernel_1764/1465479532.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa.run({"query": query, "context": context})
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** What is the primary purpose of the EU AI Act according to its introduction?


**<font color='green'>Answer:</font>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Board.
(4) AI is a f ast ev olving f amily of t echnologies that contr ibut es to a wide ar ra y of economic, environmental and societal 
benefits across the entire spectr um of industr ies and social activities. By imp roving prediction, optimising operations 
and resource allocation, and personalising digital solutions ava ilable f or individuals and org anisations, the use of AI 
can provide k ey comp etitive advantages t o under takings and suppor t socially and environmentally beneficial 
outcomes, f or exam ple in healthcare, agr iculture, f ood safety , education and training, media, spor ts, culture, 
infrastr ucture manag ement, energy , transpor t and logistics, public ser vices, secur ity , justice, resource and energy 
efficiency , en vironmental monitoring, the conser vation and restoration of biodiversity and ecosystems and climate 
ch ange mitig ation and adap tation.
(5) At the same time, depending on the circumstances regarding its specific application, use, and level of te chnological

Board.
(4) AI is a f ast ev olving f amily of t echnologies that contr ibut es to a wide ar ra y of economic, environmental and societal 
benefits across the entire spectr um of industr ies and social activities. By imp roving prediction, optimising operations 
and resource allocation, and personalising digital solutions ava ilable f or individuals and org anisations, the use of AI 
can provide k ey comp etitive advantages t o under takings and suppor t socially and environmentally beneficial 
outcomes, f or exam ple in healthcare, agr iculture, f ood safety , education and training, media, spor ts, culture, 
infrastr ucture manag ement, energy , transpor t and logistics, public ser vices, secur ity , justice, resource and energy 
efficiency , en vironmental monitoring, the conser vation and restoration of biodiversity and ecosystems and climate 
ch ange mitig ation and adap tation.
(5) At the same time, depending on the circumstances regarding its specific application, use, and level of te chnological

them are SMEs.
5. By 2 August 2028, the Commission shall evaluate the functioning of the AI Offi ce, whether the AI Office has been 
given sufficient powers and comp etences to fulf il its tasks, and whether it would be relevant and needed f or the proper 
imp lementation and enf orcement of this Regulation to upgrade the AI Offi ce and its enf orcement compet ences and to 
increase its resources. The Commission shall submit a repor t on its evaluation to the European Parliame nt and to the 
Council.
6. By 2 Augu st 2028 and ever y f our y ears thereaf te r , the Commission shall submit a repor t on the review of the progress 
on the development of standardisation deliverables on the energy-eff icient development of g eneral-pur pose AI models, and 
asses the need f or fur ther measures or actions, including binding measures or actions. The repor t shall be submitted to the 
European Parliament and t o the Council, and it shall be made public.
7. By 2 Au gust 2028 and ever y three y ears thereaf ter , the Commission shall evaluate the imp act and effe ctiveness of

them are SMEs.
5. By 2 Au gust 2028 and ever y three y ears thereaf ter , the Commission shall evaluate the imp act and effe ctiveness of

appropr iat e f ollow-up actions could contr ibut e to impro ving work ing conditions and ultimately sustain the 
consolidation, and inno vation path of tr ustwor thy AI in the Uni on. The European Ar tificial Intellig ence Board (the 
‘Board’) should suppor t the Commission, to promot e AI litera cy tools, public a wareness and understanding of the 
benefits, r isks , safegua rds, r ights and obliga tions in relation to the use of AI syste ms. In cooperation with the relevant 
stak eholders, the Commission and the Member States should f acilitate the drawing up of v oluntar y codes of conduct 
t o advance AI lite racy among persons dealing with the development, operation and use of AI.
(21) In order to ensure a level pla ying field and an effective protect ion of r ights and freedoms of individuals across the 
Uni on, the r ules established by this Regulation should apply to provid ers of AI syste ms in a non-discr iminat or y 
manner , ir respective of whether the y are established within the Uni on or in a third countr y , and to deplo y ers of AI



**<font color='red'>Question:</font>** What is the primary purpose of the EU AI Act according to its introduction?
Helpful 

**<font color='green'>Answer:</font>** The EU AI Act is a piece of legislation that aims to regulate the development and deployment of artificial intelligence systems in the European Union. Its primary purpose is to ensure that AI systems are developed and used in a way that respects fundamental rights, protects individuals, and promotes innovation and competitiveness in the AI sector.

Explanation: The EU AI Act is a comprehensive regulatory framework that seeks to address the challenges and opportunities presented by AI technologies. It is designed to provide a clear and consistent set of rules and principles for the development, deployment, and use of AI systems in the EU. The Act aims to ensure that AI systems are developed and used in a way that respects fundamental rights, protects individuals, and promotes innovation and competitiveness in the AI sector.

The Act covers a wide range of AI systems, including those used in areas such as healthcare, transportation, finance, and public administration. It sets out a series of obligations for AI developers, deployers, and users, including requirements for transparency, accountability, and robustness. The Act also establishes a system of oversight and enforcement mechanisms to ensure compliance with the rules and principles set out in the Act.

Overall, the EU AI Act is a significant step towards regulating the development and deployment of AI technologies in the EU. It seeks to strike a balance between promoting innovation and competitiveness in the AI sector and ensuring that AI systems are developed and used in a way that respects fundamental rights, protects individuals, and promotes a safe and secure digital environment.

The EU AI Act is a piece of legislation that aims to regulate the development and deployment of artificial intelligence systems in the European Union. The Act aims to ensure that AI systems


**<font color='magenta'>Total time:</font>** 25.11 sec.

In [44]:
query = "How does the regulation define 'high-risk AI systems,' and what areas are specifically affected?"
test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** How does the regulation define 'high-risk AI systems,' and what areas are specifically affected?


**<font color='green'>Answer:</font>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

prospective provider s of AI syste ms the possibility t o develop, train, validate and te st, where appropr iat e in real-wo rld 
conditions, an innovative AI system, pursuant to a sandbo x plan f or a limited time under regulator y super vision;
(56) ‘ AI literacy’ means skills, knowledg e and understanding that allow provider s, deplo y ers and affect ed persons, taking 
into account their respective r ights and obliga tions in the context of this Regulation, to make an inf or med deplo yment 
of AI systems, as well as to g ain awareness about the oppor tunities and r isks of AI and possible har m it can cause;
OJ L, 12.7.2024
EN
ELI: http://data.europa.eu/eli/reg/2024/1689/oj 49/144

prospective provider s of AI syste ms the possibility t o develop, train, validate and te st, where appropr iat e in real-wo rld 
conditions, an innovative AI system, pursuant to a sandbo x plan f or a limited time under regulator y super vision;
(56) ‘ AI literacy’ means skills, knowledg e and understanding that allow provider s, deplo y ers and affect ed persons, taking 
into account their respective r ights and obliga tions in the context of this Regulation, to make an inf or med deplo yment 
of AI systems, as well as to g ain awareness about the oppor tunities and r isks of AI and possible har m it can cause;
OJ L, 12.7.2024
EN
ELI: http://data.europa.eu/eli/reg/2024/1689/oj 49/144

r ights and guarantees a warded to them b y such Uni on law , including the r ights related to solely automat ed individual 
decision-making, including prof iling. Har monised r ules f or the placing on the marke t, the putting into ser vice and 
the use of AI systems established under this Regulation should f acilitate the effe ctive imp lementation and enable the 
ex ercise of the data subjects’ r ights and other remedies guaranteed under Uni on law on the protect ion of personal 
data and of other fundamental r ights.
(11) This Regulation should be without prejudice to the provisions rega rding the liability of provider s of inte r mediar y 
ser vices as set out in Regulation (EU) 2022/2065 of the European Parliament and of the Council (
15
).
(12) The notion of ‘ AI system’ in this Regulation should be clearly defined and should be closely aligned with the w ork of 
inte r national organisations w orking on AI to ensure leg al cer tainty , f acilitate inter national conve rgence and wide

r ights and guarantees a warded to them b y such Uni on law , including the r ights related to solely automat ed individual 
decision-making, including prof iling. Har monised r ules f or the placing on the marke t, the putting into ser vice and 
the use of AI systems established under this Regulation should f acilitate the effe ctive imp lementation and enable the 
ex ercise of the data subjects’ r ights and other remedies guaranteed under Uni on law on the protect ion of personal 
data and of other fundamental r ights.
(11) This Regulation should be without prejudice to the provisions rega rding the liability of provider s of inte r mediar y 
ser vices as set out in Regulation (EU) 2022/2065 of the European Parliament and of the Council (
15
).
(12) The notion of ‘ AI system’ in this Regulation should be clearly defined and should be closely aligned with the w ork of 
inte r national organisations w orking on AI to ensure leg al cer tainty , f acilitate inter national conve rgence and wide

development, as well as the potential ch ange s in the use of AI systems.
(53) It is also impor tant to clar ify that there ma y be specific cases in which AI syste ms refer red to in pre-define d areas 
specified in this Regulation do not lead to a signifi cant r isk of har m to the lega l intere sts protect ed under those areas 
because they do not materi ally inf luence the decision-making or do not har m those interests substantially . For the 
pur poses of this Regulation, an AI system that does not mat er ially inf luence the outcome of decision-making should 
be understood to be an AI syste m that does not have an im pact on the substance, and thereby the outcome, of 
decision-making, whether human or automa ted. An AI system that does not materially inf luence the outcome of 
decision-making could include situations in which one or more of the f ollo wing conditions are fulfilled. The fir st 
suc h condition should be that the AI syste m is intende d t o perfo r m a nar row procedural task, suc h as an AI system



**<font color='red'>Question:</font>** How does the regulation define 'high-risk AI systems,' and what areas are specifically affected?
Helpful 

**<font color='green'>Answer:</font>** 'High-risk AI systems' are defined in the regulation as those that "do not lead to a significant risk of harm to the legal interests protected under those areas because they do not materially influence the decision-making or do not harm those interests substantially." The specific areas affected by this definition are those specified in the regulation, including "the protection of personal data, the protection of health, the protection of children and the protection of fundamental rights." The regulation also clarifies that an AI system that does not materially influence the outcome of decision-making could include situations in which one or more of the following conditions are fulfilled: the AI system is intended to perform a narrow procedural task, such as an AI system that is used to detect and analyze images or videos for the purpose of identifying objects or people, or an AI system that is used to predict the outcome of a legal proceeding based on past cases and legal precedents.


**<font color='magenta'>Total time:</font>** 4.632 sec.

In [57]:
query = "Why is it important to have a consistent and uniform regulatory framework for AI across the European Union, and what Articles of the Treaty on the Functioning of the European Union (TFEU) form the basis of these regulations?"
test_rag(qa, query)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.




> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** Why is it important to have a consistent and uniform regulatory framework for AI across the European Union, and what Articles of the Treaty on the Functioning of the European Union (TFEU) form the basis of these regulations?


**<font color='green'>Answer:</font>** Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

capabilities should also include making use of existing resources and exper tise, in par ticular through synergies with 
str uctures built up in the cont ext of the Union level enf orcement of other law and synergies with related initiatives at 
Uni on level, suc h as the EuroHPC Joint Und er taking and the AI te sting and exper imentation facilities under the 
Digital Europe Programme.
(149) In order to f acilitate a smooth, eff ective and har monised implementation of this Regulation a Board should be 
established. The Board should ref lect the var ious interests of the AI eco-system and be comp osed of representatives 
of the Member Stat es. The Board should be responsible f or a number of advisor y tasks , including issuing opinions, 
recommendations, advice or contr ibuting to guidance on matt ers related t o the imp lementation of this Regulation, 
including on enf orcement matt ers, tec hnical specif ications or existing standards regarding the requirements 
established in this Regulation and pro viding advice t o the Commission and the Member States and their national

capabilities should also include making use of existing resources and exper tise, in par ticular through synergies with 
str uctures built up in the cont ext of the Union level enf orcement of other law and synergies with related initiatives at 
Uni on level, suc h as the EuroHPC Joint Und er taking and the AI te sting and exper imentation facilities under the 
Digital Europe Programme.
(149) In order to f acilitate a smooth, eff ective and har monised implementation of this Regulation a Board should be 
established. The Board should be responsible f or a number of advisor y tasks , including issuing opinions, 
recommendations, advice or contr ibuting to guidance on matt ers related t o the imp lementation of this Regulation, 
including on enf orcement matt ers, tec hnical specif ications or existing standards regarding the requirements 
established in this Regulation and pro viding advice t o the Commission and the Member States and their national

requirements or oblig ations laid do wn in this Regulation, including those established in Ar ticles 9, 10, 11, 12, 13, 14, 15, 
72 and 73.
C HAPTER VII
GO VERNANCE
SECTION 1
Gove r nance at Union le v el
Ar ticle 64
AI Off ice
1. The Commission shall develop Union exper tise and capabilities in the field of AI through the AI Off ice.
2. Member Stat es shall facilitat e the tasks entr uste d t o the AI Off ice, as ref lecte d in this Regulation.
Ar ticle 65
Es tablishment and s tr ucture of the European Ar tif icial Intelligence Board
1. A European Ar tif icial Intellig ence Board (the ‘Board’) is hereb y established.
2. The Board shall be comp osed of one representative per Member Stat e. The European Data Protection Super visor shall 
par ticipate as obser ver . The AI Office shall also attend the Board’s meetings, without taking par t in the v otes. Other national 
and Uni on author ities, bodies or exper ts ma y be invit ed to the meetings by the Board on a case by case basis, where the 
issues discussed are of relevance f or them.

requirements or oblig ations laid do wn in this Regulation, including those established in Ar ticles 9, 10, 11, 12, 13, 14, 15, 
72 and 73.
C HAPTER VII
GO VERNANCE
SECTION 1
Gove r nance at Union le v el
Ar ticle 64
AI Off ice
1. Other national 
and Uni on author ities, bodies or exper ts ma y be invit ed to the meetings by the Board on a case by case basis, where the 
issues discussed are of relevance f or them.

Section 2, af ter all other reasonable means to ver ify conf or mity hav e been exhaust ed and hav e pro ven to be 
insuffic ient, and upon a reasoned request, the notif ied body shall also be grante d access to the training and trained 
models of the AI system, including its relevant parameters. Such access shall be subject t o existing Union law on the 
protect ion of intellectual proper ty and trade secrets.
4.6. The decision of the notif ied body shall be notif ied t o the provider or its author ised representative. The notif ication 
shall contain the conclusions of the assessment of the t echnical documentation and the reasoned assessment 
decision.
Where the AI system is in conf or mity with the requirements set out in Chapt er III, Section 2, the notif ied body shall 
issue a Union te chnical documentation assessment cer tif icat e. The cer tif icat e shall indicate the name and address of 
the provider , the conclusions of the examination, the conditions (if any) f or its validity and the data necessar y f or the 
identifica tion of the AI syste m.



**<font color='red'>Question:</font>** Why is it important to have a consistent and uniform regulatory framework for AI across the European Union, and what Articles of the Treaty on the Functioning of the European Union (TFEU) form the basis of these regulations?
Helpful 

**<font color='green'>Answer:</font>** It is important to have a consistent and uniform regulatory framework for AI across the European Union because it ensures that all Member States are subject to the same rules and regulations when it comes to AI, which helps to promote fairness and equality in the EU. Additionally, it ensures that businesses operating in the EU can have confidence that they are operating within a consistent regulatory environment, which can help to reduce administrative burdens and facilitate trade. The basis of these regulations is found in the TFEU, particularly in Articles 114 and 115, which provide the legal basis for the EU to adopt measures to harmonize national laws in the area of AI.


**<font color='magenta'>Total time:</font>** 3.272 sec.

### Document Sources

To check the document sources for the last query, we follow these steps:

1. **Run a Similarity Search:**  
   Perform a similarity search in the vector database based on the query.

2. **Iterate Through Retrieved Documents:**  
   Loop through the documents returned by the similarity search.

3. **Extract and Display Metadata:**  
   For each document:
   - Print the document source from its metadata.
   - Display the page content of the document.

This process helps identify the origin and content of the documents retrieved during the query.


In [35]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Query: How many recitals are included in the first section of the EU AI Act document? Please provide the last numbered recital for an exact count
Retrieved documents: 4
Source:  ./EU_AI_Act.pdf
Text:  6(3) and the Commission guidelines.
OJ L, 12.7.2024
EN
ELI: http://data.europa.eu/eli/reg/2024/1689/oj 107/144 

Source:  ./EU_AI_Act.pdf
Text:  them are SMEs.
5. By 2 August 2028, the Commission shall evaluate the functioning of the AI Offi ce, whether the AI Office has been 
given sufficient powers and comp etences to fulf il its tasks, and whether it would be relevant and needed f or the proper 
imp lementation and enf orcement of this Regulation to upgrade the AI Offi ce and its enf orcement compet ences and to 
increase its resources. The Commission shall submit a repor t on its evaluation to the European Parliame nt and to the 
Council.
6. By 2 Augu st 2028 and ever y f our y ears thereaf te r , the Commission shall submit a repor t on the review of the progress 
on the development 

## Create Error Rating Data-sheet

We should now create a dataframe where we can compare the results of our model with and without RAG. That way we can compare the results and use them to calculate an Error Rating (ER) to verify our model and the impact of RAG. This will further be evaulated in the report. 

In [35]:
import pandas as pd

df = pd.read_csv('data/Updated_Questions_Answers.csv')
df.head(5)

,Questions,True Answer,No RAG Answer,RAG Answer,No RAG Similarity,No RAG Error,RAG Similarity,RAG Error
0,What is the purpose of the EU AI regulation act?,The purpose of this Regulation is to improve t...,What are the main challenges and opportunities...,The EU AI regulation act's purpose is to impro...,0.342695,0.657305,0.918667,0.081333
1,What is the primary purpose of the EU AI Act a...,The purpose is to improve the functioning of t...,NaN,NaN,NaN,NaN,NaN,NaN
2,How does the regulation define 'high-risk AI s...,High-risk AI systems are those posing signific...,NaN,NaN,NaN,NaN,NaN,NaN
3,What measures does the EU AI Act propose to en...,The regulation requires transparency measures ...,NaN,NaN,NaN,NaN,NaN,NaN
4,How does the regulation address the use of AI ...,This use is prohibited except in narrowly defi...,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_test = df.drop(index=0)
#df_test= df_test.head(5)
df_test.head(20)

,Questions,True Answer,No RAG Answer,RAG Answer,No RAG Similarity,No RAG Error,RAG Similarity,RAG Error
1,What is the primary purpose of the EU AI Act a...,The purpose is to improve the functioning of t...,NaN,NaN,NaN,NaN,NaN,NaN
2,How does the regulation define 'high-risk AI s...,High-risk AI systems are those posing signific...,NaN,NaN,NaN,NaN,NaN,NaN
3,What measures does the EU AI Act propose to en...,The regulation requires transparency measures ...,NaN,NaN,NaN,NaN,NaN,NaN
4,How does the regulation address the use of AI ...,This use is prohibited except in narrowly defi...,NaN,NaN,NaN,NaN,NaN,NaN
5,What are the obligations for deployers of AI s...,"Deployers must monitor system use, ensure inpu...",NaN,NaN,NaN,NaN,NaN,NaN
6,What prohibitions are outlined for the use of ...,The regulation prohibits AI systems that explo...,NaN,NaN,NaN,NaN,NaN,NaN
7,How does the regulation ensure that AI systems...,It aligns with GDPR and includes safeguards fo...,NaN,NaN,NaN,NaN,NaN,NaN
8,What exceptions are allowed for the use of AI ...,Exceptions include searching for missing perso...,NaN,NaN,NaN,NaN,NaN,NaN
9,What is the role of the European Artificial In...,"The Board supports implementation, advises on ...",NaN,NaN,NaN,NaN,NaN,NaN
10,How does the regulation address the integratio...,It emphasizes principles such as human oversig...,NaN,NaN,NaN,NaN,NaN,NaN


In this dataframe we have several questions towards the EU AI act that with the related ground truth answers (True Answers). 

We will make the model: 
- Loop though the questions
- Provide No RAG Answers
- Provide Answers with RAG
- Save the Dataframe for further analysis

In [29]:
def no_rag_model(tokenizer, pipeline, message):
    try:
        sequences = pipeline(
            message,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            max_length=200,
        )
        
        question = sequences[0]['generated_text'][:len(message)]
        answer = sequences[0]['generated_text'][len(message):]
        total_time = round(time() - time_start, 3)
        return f"Answer: {answer.strip()}"
    except Exception as e:
        return f"Error in test_model: {str(e)}"


def rag_model(qa_chain, query):
    try:
        response = qa_chain.invoke(query)
        total_time = round(time() - time_start, 3)
        response = clean_response(response)
        return f"Answer: {response.strip()}"
    except Exception as e:
        return f"Error in test_rag: {str(e)}"


In [37]:
df = df_test

for index, row in df.iterrows():
    question = row["Questions"]
    
    # No RAG Answer
    time_start = time()
    no_rag_response = no_rag_model(tokenizer, query_pipeline, question)
    time_end = time()
    no_rag_time = round(time_end - time_start, 3)
    df.at[index, "No RAG Answer"] = no_rag_response
    #print(f"No RAG Response for Q{index}: {no_rag_response} (Time: {no_rag_time}s)")
    
    # RAG Answer
    time_start = time()
    rag_response = rag_model(qa_chain, question)
    time_end = time()
    rag_time = round(time_end - time_start, 3)
    df.at[index, "RAG Answer"] = rag_response
    #print(f"RAG Response for Q{index}: {rag_response} (Time: {rag_time}s)")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=1000) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the document

In [38]:
#print(df["RAG Answer"])
df.head(20)

,Questions,True Answer,No RAG Answer,RAG Answer,No RAG Similarity,No RAG Error,RAG Similarity,RAG Error
1,What is the primary purpose of the EU AI Act a...,The purpose is to improve the functioning of t...,Answer: What are the main objectives of the Ac...,Answer: The primary purpose of the EU AI Act a...,NaN,NaN,NaN,NaN
2,How does the regulation define 'high-risk AI s...,High-risk AI systems are those posing signific...,Answer: What are the key requirements of the n...,Answer: The regulation defines 'high-risk AI s...,NaN,NaN,NaN,NaN
3,What measures does the EU AI Act propose to en...,The regulation requires transparency measures ...,Answer: How will the AI Act address transparen...,"Answer: By 2 August 2028, the Commission shall...",NaN,NaN,NaN,NaN
4,How does the regulation address the use of AI ...,This use is prohibited except in narrowly defi...,Answer: This is a question that has been raise...,Answer: In order to protect the right of other...,NaN,NaN,NaN,NaN
5,What are the obligations for deployers of AI s...,"Deployers must monitor system use, ensure inpu...",Answer: What are the obligations for developer...,Answer: The obligations for deployers of AI sy...,NaN,NaN,NaN,NaN
6,What prohibitions are outlined for the use of ...,The regulation prohibits AI systems that explo...,Answer: What prohibitions are outlined for the...,Answer: Aside from the many beneficial uses of...,NaN,NaN,NaN,NaN
7,How does the regulation ensure that AI systems...,It aligns with GDPR and includes safeguards fo...,Answer: The EU General Data Protection Regulat...,Answer: The regulation ensures that AI systems...,NaN,NaN,NaN,NaN
8,What exceptions are allowed for the use of AI ...,Exceptions include searching for missing perso...,Answer: The EU AI Act introduces a broad range...,Answer: The main principles of the EU AI Act a...,NaN,NaN,NaN,NaN
9,What is the role of the European Artificial In...,"The Board supports implementation, advises on ...",Answer: What are the main tasks of the Europea...,Answer: The European Artificial Intelligence B...,NaN,NaN,NaN,NaN
10,How does the regulation address the integratio...,It emphasizes principles such as human oversig...,Answer: What are the main challenges to be ove...,Answer: This regulation addresses the integrat...,NaN,NaN,NaN,NaN


In [39]:
df.to_csv("test_Data.csv", index=False)


### Conclusions

We utilized **LangChain**, **ChromaDB**, and **Llama3** as the LLM to build a **Retrieval-Augmented Generation (RAG)** solution. For testing purposes, we worked with the **EU AI Act (2023)**.

#### Key Findings
- The RAG model successfully provided accurate answers to questions about the EU AI Act.

#### Areas for Improvement
To enhance the solution, we plan to:
1. **Optimize the Embeddings:** Improve the quality and relevance of the embeddings used in the retrieval process.
2. **Implement Advanced RAG Schemes:** Explore and apply more complex RAG architectures for better performance and flexibility.
